In [ ]:
from database import *
try:
    db.create_all()
    print("Database successfully created.")
except Exception as e:
    print("Exception occurred.",e)

In [ ]:
from flask import Flask, render_template, redirect, url_for, session, request, flash
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import UserMixin, LoginManager, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from datetime import datetime
import pandas as pd

bcrypt = Bcrypt(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

################################################################################################################

# DEFINE FORMS

################################################################################################################


class RegisterForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(), Email(message="Invalid Email"), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")


class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("Login")

class NewTaskForm(FlaskForm):
    task = StringField("Task", validators=[InputRequired(), Length(max=100)])
    ddl = StringField("Deadline", validators=[InputRequired(), Length(max=25)])
    submit = SubmitField("Submit")

    
################################################################################################################

# FUNDAMENTAL ROUTES

################################################################################################################


@app.route('/index')
@app.route('/')
def home():
    return render_template('index.html', title='Home')


@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)

@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        
        if user.username == 'admin':
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("mytask"))
        elif user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("dashboard"))
        
        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)

@app.route('/logout', methods=["GET","POST"])
@login_required
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))   

@app.route("/park")
def park():    
    df = pd.read_csv("park.csv")
    list = df.to_dict('records')
    return render_template('park.html', entries = list)

@app.route("/museum")
def museum():    
    df = pd.read_csv("art_museum.csv",error_bad_lines = False, encoding='gbk',skiprows = 2)
    df.drop('序号', axis='columns', inplace=True)
    df.rename(columns={'序号.1':'序号'}, inplace=True)
    df1 = df.dropna(axis=0)
    list = df1.to_dict('records')
    return render_template('museum.html', entries = list)

################################################################################################################

# NORMAL USERS ROUTES

################################################################################################################

@app.route('/dashboard', methods=["GET","POST"])
@login_required
def dashboard():
    posts = Blog.query.filter_by(author=str(current_user)).all()
    return render_template('dashboard.html',posts=posts, title='Dashboard')

@app.route('/add')
@login_required
def add():
    return render_template('add.html')

@app.route('/addpost', methods=['GET','POST'])
@login_required
def addpost():
    post = Blog(title=request.form.get('title'), author=str(current_user), date_posted=datetime.now(), content=request.form.get('content'))
    db.session.add(post)
    db.session.commit()                
    return redirect(url_for('dashboard'))

@app.route('/post/<int:post_id>')
@login_required
def post(post_id):
    post = Blog.query.filter_by(id=post_id).one()
    return render_template('post.html', post=post)

################################################################################################################

# ADMIN ROUTES

################################################################################################################

@app.route('/mytask', methods=['GET','POST'])
@login_required
def mytask():
    tasks = Task.query.filter_by(author=str(current_user)).all()
    return render_template('mytask.html', tasks=tasks, title='My Tasks')

@app.route('/newtask', methods=['GET','POST'])
@login_required
def newtask():
    form = NewTaskForm()
    if form.validate_on_submit():
        newtask = Task(task=form.task.data, ddl=form.ddl.data, author=str(current_user))
        db.session.add(newtask)
        db.session.commit()
        return redirect(url_for('mytask'))
    return render_template('newtask.html', title='New Task', form=form)

@app.route('/delete/<int:task_id>', methods=['GET',"POST"])
@login_required
def delete(task_id):
    task = Task.query.get_or_404(task_id)
    db.session.delete(task)
    db.session.commit()
    return redirect(url_for('mytask'))

@app.route('/update/<int:task_id>', methods=['GET',"POST"])
@login_required
def update(task_id):
    updatetask = Task.query.filter_by(id=task_id).first()
    form = NewTaskForm()
    form.task.data = updatetask.task
    form.ddl.data = updatetask.ddl
    if form.validate_on_submit():
        updatetask.task = request.form.get('task')
        updatetask.ddl = request.form.get('ddl')
        db.session.commit()
        return redirect(url_for('mytask'))
    return render_template('update.html', title='Update Task', form=form)

################################################################################################################

# ERROR HANDLERS

################################################################################################################

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

################################################################################################################

# FINALLY, RUN!

################################################################################################################
    
if __name__ == '__main__':
    app.run('localhost', 9026)